In [1]:
import pandas as pd
import re

pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 120)

df = pd.read_excel('test_SM.xlsx', sheet_name='Массив')
df.columns = ['Наименование', 'Комментарий', 'Продуктовая группа', 'Группа', 'Производитель', 'ед изм']

## Поиск существительных
### Подготовка

In [2]:
# перевод всех буквенных символов в нижний регистр
names = df['Наименование'].tolist()

for i in range(len(names)):
    names[i] = names[i].lower()

In [3]:
# замена группы символов на один
def replace_all(simb,text, subs):
    return re.sub(simb, subs, text)

In [4]:
for i in range(len(names)):
    names[i] = replace_all('[-)/,(№1234567890]', names[i], ' ')

In [5]:
# встака пробела после точек и запятых
for i in range(len(names)):
    names[i] = names[i].replace('.', '. ', names[i].count('.')).replace(',', ', ', names[i].count(','))

In [6]:
# замена множества пробелов на один пробел
def clean_spaces(text):
    return re.sub(r'\s+', ' ', text)

In [7]:
for i in range(len(names)):
    names[i] = clean_spaces(names[i])

### Создание первого варианта словаря существительных
Для работы использован словарь существительных из 51 тыс. слов по словарю Ефремовой, составленый в рамках проекта Russian Nouns by Harrix

In [8]:
f = open('n_dict.txt', 'r', encoding='utf8')
dict_noun = f.read().splitlines()
f.close()

### Поиск и замена сокращений
В данном случае сокращением будет считаться последовательность буквенных символов длинной 1 или более с одной точкой в конце. Расшифровка сокращений будет присвоена вручную, сокращения, значение которых не удалось определить будут оставлены без изменений, сокращения не несущие смысла или ошибочно попавшие в список будут заменениы на пробел

In [9]:
shorts = []
pos = [] # список содержит порядковые номера строк для дальнейшего просмотра при определении значения сокращения

pattern = r'\w{1,}\.'

for i in range(len(names)):
    short_name = re.findall(pattern, names[i])
    if short_name:
        pos.append(i)
        if short_name not in shorts:
            shorts.append(short_name)

shorts_c = [item for sublist in shorts for item in sublist]

In [10]:
# просмотр строк с сокращениями
print(f"строк с сокращениями - {len(pos)} , сокращений - {len(shorts_c)}")

#for p in pos:
    #print(names[p])

строк с сокращениями - 139 , сокращений - 92


In [11]:
# замена сокращений

# назначаем перевод сокращениий вручную
shorts_mean = ['вспомогательный ', 'передний ', 'ходовой ', 'клапан ', 'ячейка', 'нижний', 'эксцентрическая ', 'уплотнитель ',
               'шестигранный ', 'гидравлический ', 'штука ', 'электро ', 'кольцевое ',
               ' ', 'поставляется ', ' ', 'упаковка ', 'штука ', ' ', ' ', 'анкерный ', 'пластина ',
               'верхний ', 'измерительный ', 'класс ', 'колосниковый ', 'анкерный ', 'пластина ', 'нижний', 'установочный ',
               'колосниковый ', 'гранной ', 'оцинкованый ', 'резиновый ', 'футеровочная ', 'предохранительный ',
               'комплект ', 'элемент ', 'индикатор ', 'индикатор ', ' ', 'гидравлический ', 'конвеер ',
               'неприрывная ', 'промежуточная ', 'ответной ', 'уплотнение ', 'регулировочные ', 'сборе ',
               'гидравлический ', ' ', 'направляющая ', 'шланг ', 'брызгозащитное ',
               'ограждение ', 'комплект ', 'колосника ', ' ', 'стальной ', 'системный ', 'чп ', 'промышленный ', 'порт ',
               'использовать ', 'оцинкованный ', ' ', ' ', 'чертежу ', 'мм ', 'пневмопривода ', 'фланец ', ' ',
               'передаточное ', 'отношение ', ' ', ' ', 'град ', 'кв ', 'торцевой ', 'уплотнитель ', ' ',
               'антиконденсатор ', 'ацетатный ', 'шестигранный ', 'головка ', 'оцинкованный ', 'класс ', 'прочности ',
               'тх ', 'электро ', 'взрыво ', 'защитный ']

dict_shorts = dict(zip(shorts_c, shorts_mean))

for i in range(len(names)):
    line = names[i].split()
    for j in line:
        for word in dict_shorts:
            if j.startswith(word):
                names[i] = names[i].replace(word, dict_shorts[word])

### Выделение существительных 
Выделение существительных из строк будет проводится в 2 этапа, так как исходный словарь может не содержать в себе специализированных технических терминов.
На первом этапе будут определены существительные, которые есть в словаре. После этого в ручную будут добавлены существительные из строк, в которых не определилось ни одного существительного.
На втором этапе будут выявлены недостающие существительные.
В строках с наименованием на латинице существительные не будут определены

In [12]:
# выделение существительных - (1 этап)
temp_list = []
noun_list=[]
for i in range(len(names)):
    line = names[i].split()
    for word in line:
        if word in dict_noun:
            temp_list.append(word)
        elif word[:-1] in dict_noun:
            temp_list.append(word)
        elif word[:-2] in dict_noun:
            temp_list.append(word)
    noun_list.append(temp_list)
    temp_list=[]

nouns = pd.Series(noun_list)
df['существительные'] = nouns.values

df.sort_values(by='существительные')

,Наименование,Комментарий,Продуктовая группа,Группа,Производитель,ед изм,существительные
847,TRELLEX ABC-EASY 25-130-2000,NaN,Конвейерные аксессуары,SN,NaN,шт,[]
134,SLIP RING,NaN,Дробилки. Запчасти,SN,NaN,шт,[]
360,Trellex PP 100-500-1500,NaN,Материалы: резина/полиуретан,SN,NaN,шт,[]
138,FLANGE PIPE,NaN,Дробилки. Запчасти,SN,NaN,шт,[]
359,Trellex PP 75-750-1500,NaN,Материалы: резина/полиуретан,SN,NaN,шт,[]
964,"ML-WASHPROTECTION 0,3L",NaN,Мел. футеровка крепеж,Temp,NaN,шт,[]
356,Trellex PP 75-600-1000,NaN,Материалы: резина/полиуретан,SN,NaN,шт,[]
699,THREADED ROD,NaN,Дробилки. Запчасти,SN,NaN,шт,[]
355,Trellex PP 75-500-1500,NaN,Материалы: резина/полиуретан,SN,NaN,шт,[]
354,Trellex PP 75-500-1000,NaN,Материалы: резина/полиуретан,SN,NaN,шт,[]


In [13]:
# добавление новых слов
nouns = pd.Series(noun_list)
#nam = pd.Series(names)
df['существительные'] = nouns.values
#df['проверка'] = nam.values

df.sort_values(by='существительные')

new_words = ['картридж', 'дампер', 'резинокерамика', 'подситник', 'проставка', 'штекер', 'гидроцилиндр',
             'лубрикатор', 'клеммник', 'переходник', 'расходомер', 'комплектующие', 'ноутбук', 'корщетка',
             'муфта', 'удлинитель', 'крыльчатка', 'лопаст', 'колосник', 'щетка','комплект', 'индикатор', 'консол',
            'уплотнение', 'уплотнитель', 'гидроцилидр' 'маслопровод', 'решетка', 'позиционер', 'планка', 'пластин',
            'гровер', 'разделитель', 'байпас']

In [14]:
# создание нового словаря с дополнениями
dict_noun.extend(new_words)

In [15]:
# выделение существительных - (2 этап)
temp_list = []
noun_list=[]
for i in range(len(names)):
    line = names[i].split()
    for word in line:
        if word in dict_noun:
            temp_list.append(word)
        elif word[:-1] in dict_noun:
            temp_list.append(word)
        elif word[:-2] in dict_noun:
            temp_list.append(word)
    noun_list.append(temp_list)
    temp_list=[]

In [16]:
# проверка результатов 

nouns = pd.Series(noun_list)
df['существительные'] = nouns.values
df['существительные'] = df['существительные'].astype(str).str[1:-1]

df

,Наименование,Комментарий,Продуктовая группа,Группа,Производитель,ед изм,существительные
0,"Фильтрующий элемент - карманный, тип AJN Long, PN-12",INFASTAUB,Дробилки. Запчасти,SN,NaN,шт,"'элемент', 'тип'"
1,"DIN-рейка перфорированная OMEGA 3F, 35х7,5 мм",Производитель DKC,Прочее ЭМП,SN,NaN,м,'рейка'
2,"Гайка 20-UNF SQ, GR8",Berco,Запчасти обогатительного оборуд.,SN,NaN,шт,'гайка'
3,"Мембрана, тип А 3/4",INFASTAUB,Дробилки. Запчасти,SN,NaN,шт,"'мембрана', 'тип'"
4,Светодиодная фара,SAE,Дробилки. Запчасти,SN,NaN,шт,'фара'
5,DIN 125 A / ISO 7089 Шайба плоская M16 (Ø17) Цинк,SVM,Дробилки. Запчасти,Temp,NaN,шт,"'шайба', 'цинк'"
6,DIN 127 B Шайба пружинная Гровер M16 Цинк,SVM,Дробилки. Запчасти,Temp,NaN,шт,"'шайба', 'гровер', 'цинк'"
7,DIN 127 B Шайба пружинная Гровер M20 Цинк,SVM,Дробилки. Запчасти,Temp,NaN,шт,"'шайба', 'гровер', 'цинк'"
8,DIN 127 B Шайба пружинная Гровер M24 Цинк,SVM,Дробилки. Запчасти,Temp,NaN,шт,"'шайба', 'гровер', 'цинк'"
9,"ШКАЛА АМПЕРМЕТРА 72Х72,200А",NaN,Schneider Electric,Schneider Electric,NaN,шт,"'шкала', 'амперметра'"


In [17]:
n0 = 0
n1 = 0
n2 = 0
for i in noun_list:
    if len(i) == 1:
        n1 +=1
    elif len(i) > 1:
        n2 +=1
    elif not len(i):
        n0 +=1
print(f"существительные определены в {n1+n2} строках \nв том числе более одного существительного в {n2} строках")

существительные определены в 882 строках 
в том числе более одного существительного в 196 строках


### Итог
1. Для большинства строк (882) были определены существительные в столбце "существительные", для строк на латинице существительные не определялись.
2. При определении существительных в строке наименования приоритет был отдан существительным в именительном падеже, так как они в большей степени отражают сущность наименования. Существительные в других падежах и или во множественном числе могли быть определены не во всех случаях из за особенностей словообразования.
3. В отдельных случаях прилагательные (белый, цветной) могли быть определены как существительные
4. Для более точного определения частей речи в наборах данных большего объема можно использовать словарь большего объема и учетом различных форм слов (корпус современного русского языка) а так же методы ML.


## Поиск дубликатов
Проводится подготовка данных аналогичная той, что использовалась для поиска существительных. Заменяются сокращения по ранее созданному словарю

In [19]:
names_dub = df['Наименование'].tolist()

# перевод символов в нижний регистр
for i in range(len(names_dub)):
    names_dub[i] = names_dub[i].lower()

# замена символов на пробел
for i in range(len(names_dub)):
    names_dub[i] = replace_all('[-)/,(]', names_dub[i], ' ')

# вставка пробела после точек и запятых
for i in range(len(names_dub)):
    names_dub[i] = names_dub[i].replace('.', '. ', names_dub[i].count('.')).replace(',', ', ', names_dub[i].count(','))

# удаление лишних пробелов 
for i in range(len(names_dub[i])):
    names_dub[i] = clean_spaces(names_dub[i])

In [20]:
# замена сокращений
for i in range(len(names_dub)):
    line = names_dub[i].split()
    for j in line:
        for word in dict_shorts:
            if j.startswith(word):
                names_dub[i] = names_dub[i].replace(word, dict_shorts[word])

In [28]:
# перечень потенциальных дубликатов
names_dubl = pd.Series(names_dub)
df['names dubl'] = names_dubl.values
df[df.duplicated(['names dubl'], keep=False)].sort_values(by='names dubl')

,Наименование,Комментарий,Продуктовая группа,Группа,Производитель,ед изм,существительные,names dubl
221,FLANGE PIPE,NaN,Дробилки. Запчасти,SN,NaN,шт,,flange pipe
138,FLANGE PIPE,NaN,Дробилки. Запчасти,SN,NaN,шт,,flange pipe
105,"Injection moulded PU trommel panel 305x305x36mm Thick/12x50mm apertures, Hi-temp/150mm",NaN,Бутары. Разгрузочные системы,SN,Multotec,шт,,injection moulded pu trommel panel 305x305x36mm thick 12x50mm apertures hi temp 150mm
106,"Injection moulded PU trommel panel 305x305x36mm Thick/12x50mm apertures, Hi-temp/150mm",NaN,Бутары. Разгрузочные системы,SN,Multotec,шт,,injection moulded pu trommel panel 305x305x36mm thick 12x50mm apertures hi temp 150mm
107,"Injection moulded PU trommel panel 305x305x36mm Thick/12x50mm apertures, Hi-temp/150mm",NaN,Бутары. Разгрузочные системы,SN,Multotec,шт,,injection moulded pu trommel panel 305x305x36mm thick 12x50mm apertures hi temp 150mm
740,PROTECTION PLATE INSTALLATION,NaN,Дробилки. Запчасти,SN,NaN,шт,,protection plate installation
619,PROTECTION PLATE INSTALLATION,NaN,Дробилки. Запчасти,SN,NaN,шт,,protection plate installation
193,PULLER,Загрузка MALP 3q16,Дробилки. Запчасти,SN,NaN,шт,,puller
127,PULLER,NaN,Дробилки. Запчасти,SN,NaN,шт,,puller
548,SHAFT,NaN,Дробилки. Запчасти,SN,NaN,шт,,shaft
